In [4]:
# загрузка 

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import json

def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

path_json = '/Users/gleb/Desktop/internerLAB/internetlab/docs/clean_docs/reg_clean/reg_over_2021-2023/2023/2023.json'

title = [i.capitalize() for i in load_data(path_json)["title"]]
content = load_data(path_json)["text"]



ModuleNotFoundError: No module named 'sklearn'

In [3]:
# tf-idf

vectorizer = TfidfVectorizer(
    lowercase=True,
    max_features=100, # тут топ частотность, т.е от самого частого к менее
    max_df=0.8, # рейтинг слов больше 80% будут игнорироваться
    min_df=3, # слова которые появляются реже 3 раз - игнорируются
    ngram_range = (2,4), # n-gram от n слова до фразы из m
    stop_words = stopwords.words('russian')
)

vectors = vectorizer.fit_transform(content)

features_name = vectorizer.get_feature_names_out() 

dense = vectors.todense() # -> matrix
denselist = dense.tolist() # -> list-matrix


In [4]:
# загрузка k-w

# финальный список 
all_keywords = []

# в данном цикле проверяется частотность слова/фраз из вектора. Если из итоговый рейтинг > 0 то это k-words
for desc in denselist:
    x=0
    keywords = list(set([]))
    for word in desc:
        if word > 0:
            keywords.append(features_name[x])
        x +=1
    all_keywords.append(keywords)    
    

# кастомное решение поиска k-w с помощью обычного метода count() !рузультаты отличаются, они не совсем верные! 

def count_keyword_occurrences(content, all_keywords):
    keyword_counts = []
    
    for i, doc in enumerate(content):
        keywords = all_keywords[i]
        temp = []
        for keyword in keywords:
            counter = doc.lower().count(keyword)
            temp.append(counter)
        keyword_counts.append(temp)

    return keyword_counts

# здесь форматирую матрицу в обычный массив
kw_counter = count_keyword_occurrences(content, all_keywords)
kw_count = []
for i in kw_counter:
    for j in i:
        kw_count.append(j)

In [5]:
# df по k-word 
data = [[title, kw] for title, content_list in zip(title, all_keywords) for kw in content_list]
df = pd.DataFrame(data, columns=['title', 'k-word'])
df['appearance']= kw_count

# df.to_csv('reg_2023_k-w_table.csv', index=False, encoding='UTF-16')

In [6]:
df.head(10)

,title,k-word,appearance
0,C_стратегия_луганская_народная_республика_2023...,единица измерения показателя значения,7
1,C_стратегия_луганская_народная_республика_2023...,задача отрасли,7
2,C_стратегия_луганская_народная_республика_2023...,задача отрасли наименование,7
3,C_стратегия_луганская_народная_республика_2023...,задача отрасли наименование проекта,7
4,C_стратегия_луганская_народная_республика_2023...,измерения показателя значения,7
5,C_стратегия_луганская_народная_республика_2023...,измерения показателя значения показателя,7
6,C_стратегия_луганская_народная_республика_2023...,исполнительные органы государственной,3
7,C_стратегия_луганская_народная_республика_2023...,используемые сцт,7
8,C_стратегия_луганская_народная_республика_2023...,используемые сцт роль,7
9,C_стратегия_луганская_народная_республика_2023...,используемые сцт роль региона,7


In [2]:
import matplotlib.pyplot as plt
import numpy as np

idf = vectorizer.idf_

mean_idf = np.mean(idf)

# Строим гистограмму
plt.hist(idf, bins=20, color='skyblue', edgecolor='black', density=True)
plt.axvline(mean_idf, color='red', linestyle='dashed', linewidth=1)
plt.xlabel('TF-IDF Value')
plt.ylabel('Frequency')
plt.title('TF-IDF Distribution')
plt.show()


NameError: name 'vectorizer' is not defined